**Реалізація каунтера з використанням PostgreSQL**

Для створення бази даних використовується наступний docker compose файл:  
  ```
services:  
  db:  
    image: postgres:14.1-alpine  
    container_name: postgres_db  
    environment:  
      - POSTGRES_USER=${USERNAME}  
      - POSTGRES_PASSWORD=${PASSWORD}  
      - POSTGRES_DB=${DATABASENAME}  
    ports:  
      - "5432:5432"  
  ```
Щоб викоритовувати змінні середовища потрібно створити .env файл в якому зазначити  USERNAME, PASSWORD, DATABASENAME.  
Для виконання завдання потрібно створити відповідну таблицю. Це можна зробити виконавши наступні кроки:  
  ```
psql -h localhost -p 5432 -d counter  
  
CREATE TABLE user_counter (  
    user_id INTEGER PRIMARY KEY,  
    counter INTEGER,  
    version INTEGER  
);  
  
INSERT INTO user_counter (user_id, counter, version) VALUES (1, 0, 0);  
INSERT INTO user_counter (user_id, counter, version) VALUES (2, 0, 0);  
INSERT INTO user_counter (user_id, counter, version) VALUES (3, 0, 0);  
INSERT INTO user_counter (user_id, counter, version) VALUES (4, 0, 0);  
```



Імпорт необхідних бібліотек:

In [13]:
import threading
import psycopg2
import time
from os import getenv
from dotenv import load_dotenv, dotenv_values 
load_dotenv("./.env")

True

Конфігурація для підключення до бази даних

In [14]:
db_config = {
    'dbname': getenv("DATABASENAME"),
    'user': getenv("USERNAME"),
    'password': getenv("PASSWORD"),
    'host': 'localhost',
    'port': 5432
}
thread_count = 10
iter_count = 10000

1. Lost-update (реалізація що втрачатиме значення)

In [15]:
def increment_counter(iter_count):
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    for _ in range(iter_count):
        cursor.execute("SELECT counter FROM user_counter WHERE user_id = 1")
        counter = cursor.fetchone()[0]
        counter += 1
        cursor.execute("UPDATE user_counter SET counter = %s WHERE user_id = %s", (counter, 1))
        conn.commit()
    cursor.close()
    conn.close()

threads = []
start_time = time.time()

for _ in range(thread_count):
    thread = threading.Thread(target=increment_counter, args=[iter_count])
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

total_time = time.time() - start_time
print(f"Загальний час виконання: {total_time:.2f} секунд")

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
cursor.execute("SELECT counter FROM user_counter WHERE user_id = 1")
final_counter = cursor.fetchone()[0]
print(f"Кінцеве значення лічильника: {final_counter} (Очікувалось: {thread_count * iter_count})")
cursor.close()
conn.close()

Загальний час виконання: 471.65 секунд
Кінцеве значення лічильника: 23598 (Очікувалось: 100000)


2. In-place update

In [16]:
def in_place_update(iter_count):
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    for _ in range(iter_count):
        cursor.execute("UPDATE user_counter SET counter = counter + 1 WHERE user_id = %s", (2,))
        conn.commit()
    cursor.close()
    conn.close()

threads = []
start_time = time.time()

for _ in range(thread_count):
    thread = threading.Thread(target=in_place_update, args=[iter_count])
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

total_time = time.time() - start_time
print(f"Загальний час виконання: {total_time:.2f} секунд")

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
cursor.execute("SELECT counter FROM user_counter WHERE user_id = 2")
final_counter = cursor.fetchone()[0]
print(f"Кінцеве значення лічильника: {final_counter} (Очікувалось: {thread_count * iter_count})")
cursor.close()
conn.close()

Загальний час виконання: 230.07 секунд
Кінцеве значення лічильника: 100000 (Очікувалось: 100000)


3. Row-level locking

In [17]:
def row_level_locking(iter_count):
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    for _ in range(iter_count):
        conn.autocommit = False
        try:
            cursor.execute("SELECT counter FROM user_counter WHERE user_id = 3 FOR UPDATE")
            counter = cursor.fetchone()[0]
            counter += 1
            cursor.execute("UPDATE user_counter SET counter = %s WHERE user_id = %s", (counter, 3))
            conn.commit()
        except Exception as e:
            conn.rollback()
            print(f"Error: {e}")
    cursor.close()
    conn.close()

threads = []
start_time = time.time()

for _ in range(10):
    thread = threading.Thread(target=row_level_locking, args=[iter_count])
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

total_time = time.time() - start_time
print(f"Загальний час виконання: {total_time:.2f} секунд")

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
cursor.execute("SELECT counter FROM user_counter WHERE user_id = 3")
final_counter = cursor.fetchone()[0]
print(f"Кінцеве значення лічильника: {final_counter} (Очікувалось: {thread_count * iter_count})")
cursor.close()
conn.close()

Загальний час виконання: 356.26 секунд
Кінцеве значення лічильника: 100000 (Очікувалось: 100000)


4. Optimistic concurrency control

In [18]:
def optimistic_concurrency_control(iter_count):
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    for _ in range(iter_count):
        while True:
            try:
                conn.autocommit = False
                cursor.execute("SELECT counter, version FROM user_counter WHERE user_id = 4")
                row = cursor.fetchone()
                if row is None:
                    print("Запис з user_id = 4 не знайдено.")
                    break
                counter, version = row
                new_counter = counter + 1
                new_version = version + 1
                cursor.execute("""
                    UPDATE user_counter
                    SET counter = %s, version = %s
                    WHERE user_id = %s AND version = %s
                """, (new_counter, new_version, 4, version))
                if cursor.rowcount > 0:
                    conn.commit()
                    break 
                else:
                    conn.rollback()
            except Exception as e:
                conn.rollback()
                print(f"Помилка: {e}")
            finally:
                conn.autocommit = True
    cursor.close()
    conn.close()

start_time = time.time()

threads = []
for _ in range(10):
    thread = threading.Thread(target=optimistic_concurrency_control, args=[iter_count])
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

total_time = time.time() - start_time
print(f"Загальний час виконання: {total_time:.2f} секунд")

conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
cursor.execute("SELECT counter FROM user_counter WHERE user_id = 4")
final_counter = cursor.fetchone()[0]
print(f"Кінцеве значення каунтера: {final_counter} (Очікувалось: {thread_count * iter_count})")
cursor.close()
conn.close()

Загальний час виконання: 959.16 секунд
Кінцеве значення каунтера: 100000 (Очікувалось: 100000)
